# Log Data Example

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.carwatch_logs import LogData
import biopsykit.carwatch_logs as cl

import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
log_path = Path("../example_data/log_data")

## Loading/Saving

Log files can either be imported by (1) loading the `.zip` files obtained by the *CARWatch App* or (2) by loading a subject folder with `.csv` files.

For (1), the file structure is assumed to look like:
```
. (base folder)
├── logs_ABC12.zip
├── logs_DEF34.zip
├── logs_GHI56.zip
├── logs_JKL78.zip
...
```

For (2), the file structure is assumed to look like:
```
. (base folder)
├── ABC12/
│   ├── carwatch_abc12_20200101.csv
│   ├── carwatch_abc12_20200102.csv
│   └── carwatch_abc12_20200103.csv
├── DEF34/
│   ├── carwatch_def34_20200201.csv
│   ├── carwatch_def34_20200202.csv
│   └── carwatch_def34_20200203.csv
...
```

### Load Single Subject

Example Log Data:

In [ ]:
bp.example_data.get_log_data_example()

In [ ]:
# Load zip file:
# logs = bp.log_data.io.load_log_one_subject(log_path.joinpath("logs_AB12C.zip"))
# Load folder:
logs = cl.io.load_log_one_subject(log_path.joinpath("DE34F"))
logs

### Save Log Data as csv

In [ ]:
cl.io.save_log_data(logs, log_path)

### Load All Subjects

Load Log files stored in subfolders and concatenate them into one large dataframe:

In [ ]:
df_logfiles = cl.io.load_logs_all_subjects(log_path, has_subfolder=True)

# If Log files are zip files (or were already exported as combined csv files), you can load them like this:
#df_logfiles = bp.log_data.io.load_logs_all_subjects(log_path, has_subfolder=False)

In [ ]:
df_logfiles

### Save All Subjects

In [ ]:
cl.io.save_log_data(df_logfiles, log_path.joinpath("export_logs.csv"))

### Create `LogData` object

The `LogData` object can help to better handle log data

In [ ]:
# select one subject using dataframe cross-select
df = df_logfiles.xs('DE34F')

logdata = LogData(df)

In [ ]:
logdata

## Filtering/Selecting

### Filter by Action

In [ ]:
cl.utils.get_logs_for_action(logdata, 'barcode_scanned')

### Extract Extras from Log Actions

In [ ]:
# returns extras for the first log entry with specific action
cl.utils.get_extras_for_log(logdata, 'barcode_scanned')

### Filter by Date

In [ ]:
cl.utils.get_logs_for_date(logdata, '2019-12-06')

### Split Log Data into different Nights

In [ ]:
list_nights = cl.utils.split_nights(logdata)

In [ ]:
list_nights[1]

## Log Statistics

In [ ]:
log_stats = cl.LogStatistics(log_path)

### Android Versions

In [ ]:
log_stats.android_versions()

In [ ]:
fig, ax = plt.subplots()
log_stats.get_plot('android_version', ax=ax)

### Smartphone Manufacturer

In [ ]:
log_stats.manufacturer()

In [ ]:
fig, ax = plt.subplots()
log_stats.get_plot('manufacturers', ax=ax)

### Recording Days

In [ ]:
log_stats.days()

In [ ]:
fig, ax = plt.subplots()
log_stats.get_plot('days', ax=ax)

## Widgets

In [ ]:
out_logfile = widgets.Output()

@out_logfile.capture(clear_output=True)
def callback_log_file_subject(change):
    subject_id = change['new']
    display(df_logfiles.xs(subject_id))

In [ ]:
from biopsykit.carwatch_logs.widgets import log_file_subject_dropdown
log_file_subject_dropdown(log_path, input_type='folder', value_type="subject_id", callback=callback_log_file_subject)

In [ ]:
out_logfile

In [ ]:
out_action = widgets.Output()

@out_action.capture(clear_output=True)
def callback_log_action(change):
    display(change['new'])

In [ ]:
from biopsykit.carwatch_logs.widgets import action_dropdown_widget
action_dropdown_widget(df_logfiles.xs('DE34F'), callback=callback_log_action)

In [ ]:
out_action

In [ ]:
out_day = widgets.Output()

@out_day.capture(clear_output=True)
def callback_log_day(change):
    display(change['new'])

In [ ]:
from biopsykit.carwatch_logs.widgets import day_dropdown_widget
day_dropdown_widget(df_logfiles.xs('DE34F'), callback=callback_log_day)

In [ ]:
out_day